In [1]:
import copy
import json
import os
import numpy
from typing import Any, Callable, Dict, List, Literal, Optional, Tuple, Union

from openai import BadRequestError

import autogen
from autogen import UserProxyAgent, AssistantAgent
from autogen import config_list_from_json
from autogen.agentchat import Agent
from autogen.agentchat.contrib.agent_optimizer import AgentOptimizer
from autogen.agentchat.contrib.math_user_proxy_agent import MathUserProxyAgent
from autogen.code_utils import extract_code
from autogen.math_utils import get_answer

## Data

In [2]:
def read_file(url):
    file = open(url, "r")
    data = file.read().split('\n')
    return data

In [3]:
phoMT_dev_en = read_file("data/PhoMT/detokenization/dev/dev.en")
phoMT_dev_vi = read_file("data/PhoMT/detokenization/dev/dev.vi")
phoMT_test_en = read_file("data/PhoMT/detokenization/test/test.en")
phoMT_test_vi = read_file("data/PhoMT/detokenization/test/test.vi")
phoMT_train_en = read_file("data/PhoMT/detokenization/train/train.en")
phoMT_train_vi = read_file("data/PhoMT/detokenization/train/train.vi")

In [4]:
phoMT_dev_en[0] = phoMT_dev_en[0][1:]

In [5]:
phoMT_dev_envi = [];
for index in range(len(phoMT_dev_en)):
    phoMT_dev_envi.append({"question":phoMT_dev_en[index], "answer": phoMT_dev_vi[index]})
phoMT_test_envi = [];
for index in range(len(phoMT_test_en)):
    phoMT_test_envi.append({"question": phoMT_test_en[index], "answer": phoMT_test_vi[index]})
phoMT_train_envi = [];
for index in range(len(phoMT_train_en)):
    phoMT_train_envi.append({"question": phoMT_train_en[index], "answer": phoMT_train_vi[index]})
# phoMT_dev_envi

In [6]:
phoMT_dev_envi[0]["question"]

'Hurricane Dorian, one of the most powerful storms ever recorded in the Atlantic Ocean, made landfall as a Category 5 storm on Great Abaco Island in the northern Bahamas on Sunday morning, September 1, 2019.'

## Benchmark

In [7]:
from comet import download_model, load_from_checkpoint

# Choose your model from Hugging Face Hub
# model_path = download_model("Unbabel/XCOMET-XL")
# or for example:
# model_path = download_model("Unbabel/wmt22-comet-da")

# Load the model checkpoint:
model = load_from_checkpoint('./XCOMET-XL/checkpoints/model.ckpt')

/home/kaylous/workspace/ics/llms/.venv/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Encoder model frozen.
/home/kaylous/workspace/ics/llms/.venv/lib/python3.11/site-packages/pytorch_lightning/core/saving.py:195: Found keys that are not in the model state dict but in the checkpoint: ['encoder.model.embeddings.position_ids']


In [8]:
data = [
    {
        "src": "Boris Johnson teeters on edge of favour with Tory MPs", 
        "mt": "Boris Johnson ist bei Tory-Abgeordneten völlig in der Gunst", 
        "ref": "Boris Johnsons Beliebtheit bei Tory-MPs steht auf der Kippe"
    }
]
model_output = model.predict(data, batch_size=8, gpus=1)
# Segment-level scores
# System-level score
# Score explanation (error spans)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
Predicting DataLoader 0: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:32<00:00, 32.74s/it]


In [9]:
model_output.scores

[0.45751163363456726]

In [10]:
model_output.system_score

0.45751163363456726

In [11]:
model_output.metadata.error_spans

[[{'text': 'ist bei',
   'confidence': 0.40954869985580444,
   'severity': 'critical',
   'start': 13,
   'end': 21},
  {'text': 'Abgeordnete',
   'confidence': 0.27366378903388977,
   'severity': 'major',
   'start': 27,
   'end': 38},
  {'text': 'völlig in der Gunst',
   'confidence': 0.5219234228134155,
   'severity': 'critical',
   'start': 39,
   'end': 59}]]

In [8]:
def get_score(src, ans, res):
    data = [
        {
            "src": src,
            "mt" : res,
            "ref": ans
        }
    ]
    return model.predict(data, batch_size=8, gpus=1).system_score

## Agent init

### Agents declarations

In [9]:
llm_config = {
    "config_list": [
        {
            "model": "llama3",
            "base_url": "http://localhost:11434/v1",
            "api_key": "ollama",
        }
    ]
}

PromptGenerator = autogen.UserProxyAgent(
    name="PromptGenerator",
    system_message="You are a prompt engineer",
    human_input_mode = "NEVER",
    code_execution_config=False,
    llm_config=llm_config,
)
LLM = autogen.AssistantAgent(
    name="LLM",
    system_message="You are a helpful assistant",
    llm_config=llm_config,
)

# user_proxy = autogen.UserProxyAgent(
#     name="Userproxyagent",
#     human_input_mode="NEVER",
#     code_execution_config={"work_dir": "_output", "use_docker": False},
# )

In [12]:
output = PromptGenerator.initiate_chat(
    recipient = LLM,
    max_turns = 1,
    message = "Translate this sentence to Vietnamese, say no more than necessary, and no notes: " + phoMT_dev_envi[0]["question"],
    answer = phoMT_dev_envi[0]["answer"]
)

PromptGenerator (to LLM):

Translate this sentence to Vietnamese, say no more than necessary, and no notes: Hurricane Dorian, one of the most powerful storms ever recorded in the Atlantic Ocean, made landfall as a Category 5 storm on Great Abaco Island in the northern Bahamas on Sunday morning, September 1, 2019.

--------------------------------------------------------------------------------
LLM (to PromptGenerator):

Cơn bão Dorian, một trong những cơn bão mạnh nhất từng được ghi nhận trên Đại Tây Dương, đã đổ bộ lên đảo Abaco lớn ở phía bắc Bahamas với sức gió Category 5 vào sáng chủ nhật ngày 1 tháng 9 năm 2019.

--------------------------------------------------------------------------------


In [63]:
output.summary

'Cơn bão Dorian, một trong những cơn bão mạnh nhất từng được ghi nhận trên Đại Tây Dương, đã đổ bộ lên đảo Abaco lớn ở phía bắc Bahamas với sức gió Category 5 vào sáng chủ nhật ngày 1 tháng 9 năm 2019.'

In [64]:
get_score(phoMT_dev_envi[0]["question"], phoMT_dev_envi[0]["answer"], output.summary)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
Predicting DataLoader 0: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:06<00:00,  6.11s/it]


0.8859372138977051

### Custom UserProxyAgent

In [41]:
def is_termination_msg_mathchat(message):
    """Check if a message is a termination message."""
    if isinstance(message, dict):
        message = message.get("content")
        if message is None:
            return False
    if message.rstrip().find("TERMINATE") >= 0:
        return True
    return False


class JudgeProxyAgent(UserProxyAgent):
    MAX_CONSECUTIVE_AUTO_REPLY = 10
    DEFAULT_REPLY_TEMPLATE = "Generate a response more closely resembling the style, detail, and tone of the provided answer. Focus on specifying key elements to capture the nuances of this answer effectively. The answer: "
    PROMPTS = """Translate a sentence from English to Vietnamese. Produce an accurate, context-sensitive translation that maintains the tone and meaning of the original sentence. Ensure that the output sounds natural for native Vietnamese speakers.
    Respond only with the requested output. Do not include any explanations, introductions, follow-up remarks, or additional feedback. Provide exactly and only what is specified in the task.
    The text:
    """

    def __init__(
        self,
        name: Optional[str] = "JudgeChatAgent",
        # is_termination_msg: Optional[Callable[[Dict], bool]] = is_termination_msg_mathchat,
        human_input_mode: Literal["ALWAYS", "NEVER", "TERMINATE"] = "NEVER",
        # default_auto_reply: Optional[Union[str, Dict, None]] = DEFAULT_REPLY,
        # max_invalid_q_per_step=3,
        **kwargs,
    ):
        super().__init__(
            name=name,
            # is_termination_msg=is_termination_msg,
            human_input_mode=human_input_mode,
            # default_auto_reply=default_auto_reply,
            # max_invalid_q_per_step=max_invalid_q_per_step,
            **kwargs,
        )
        self.register_reply(
            trigger=autogen.ConversableAgent, reply_func=JudgeProxyAgent.generate_feedback, position=0
        )
        self.register_reply(
            trigger=autogen.ConversableAgent, reply_func=JudgeProxyAgent._check_final_result, position=0
        )
        self.max_function_call_trial = 3
        self.query = None
        self._answer = None
        self.is_correct = None

    def initiate_chat(
        self,
        answer,
        recipient,
        silent: Optional[bool] = False,
        max_turns = 3,
        **context,
    ):
        self.query = context["message"]
        self._answer = answer
        self.llm_config["context"] = {"question": self.query, "answer": answer}
        # self.DEFAULT_REPLY = self.DEFAULT_REPLY_TEMPLATE + answer
        self.is_correct = None
        self.max_function_call_trial = max_turns

        self._prepare_chat(recipient, True)
        error_message = None
        try:
            prompt = self.PROMPTS + context['message']
            self.send(prompt, recipient, silent=silent)
        except BadRequestError as e:
            error_message = str(e)
            self.is_correct = 0
            print("error information: {}".format(error_message))

        recipient.reset()
        self.is_correct = copy.deepcopy(self.is_correct)
        result = self.is_correct
        # print("Check self.is_correct")
        # print(result)
        # print(str(result))
        # print("End check")
        self._reset()
        # print(recipient.chat_messages_for_summary(self))
        # print(self.chat_messages_for_summary(recipient))
        # print(recipient.get_chat_results())
        # print(self.get_chat_results())
        return result

    # reply = self.generate_reply(messages=self.chat_messages[sender], sender=sender

    def receive(
        self,
        message: Union[Dict, str],
        sender: Agent,
        request_reply: Optional[bool] = None,
        silent: Optional[bool] = False,
    ):
        self._process_received_message(message, sender, silent)
        if request_reply is False or request_reply is None and self.reply_at_receive[sender] is False:
            return
        self.max_function_call_trial = self.max_function_call_trial - 1
        if (self.max_function_call_trial <= 0):
            self.max_function_call_trial = 0
            return
        reply = self.generate_reply(messages=self.chat_messages[sender], sender=sender)
        if reply is not None:
            self.send(reply, sender, silent=silent)

    def generate_feedback(
        self,
        messages: Optional[List[Dict]] = None,
        sender: Optional[autogen.Agent] = None,
        config: Optional[Any] = None,
    ):
        # self.max_function_call_trial = self.max_function_call_trial - 1
        # if (self.max_function_call_trial <= 0):
        #     self.max_function_call_trial = 0
        #     return False, None
        messages = messages[-1]
        if isinstance(messages, dict):
            # messages = messages.get("content")
            if messages.get("content") is None:
                return False, None
        reply_prompt = f'Analyze the original sentence: {self.query}, the expected Vietnamese translation: {self._answer}, and the generated translation: {messages.get("content")}. Identify the differences between {messages.get("content")} and {self._answer}, and provide guidance to improve the translation so it aligns more closely with {self._answer}. Focus on preserving meaning, tone, style, and naturalness in Vietnamese while addressing any discrepancies.'

        toned_message = messages
        default_prompt_for_trimming = "Respond only with the requested output. Do not include any explanations, introductions, follow-up remarks, or additional feedback. Provide exactly and only what is specified in the task."
        toned_message["content"] = reply_prompt + ". " + default_prompt_for_trimming
        state, oai_reply = self.generate_oai_reply(
            messages = [toned_message],
            sender = sender,
            config = config
        )
        if state:
            return True, oai_reply
        return False, None
        
    def _check_final_result(
        self,
        messages: Optional[List[Dict]] = None,
        sender: Optional[autogen.Agent] = None,
        config: Optional[Any] = None,
    ):
        messages = messages[-1]
        if isinstance(messages, dict):
            messages = messages.get("content")
            if messages is None:
                return False, None
            if (messages.find("\n") >= 0):
                print("Response longer than expected?\n" + messages)
                # messages = messages.split("\n")[0]

        temp_score = get_score(self.query, messages, self._answer)
        self.is_correct = messages
        print("Score: " + str(temp_score))
        if (temp_score >= 0.9):
            return True, "The result is passable. Please reply me with the same answer as before."
        return False, None

    def _reset(self):
        # super()._reset()
        self.max_function_call_trial = 0
        self.is_correct = None
        self.query = None
        self._answer = None

In [42]:
llm_config_judge = {
    "config_list": [
        {
            "model": "llama3",
            "base_url": "http://localhost:11434/v1",
            "api_key": "ollama",
        }
    ],
    # "prompt": 'Analyze the original sentence {question}, the expected translation {answer}, and the generated response. Provide advice on how to guide the model toward producing translations more closely aligned with {answer}. Focus on enhancing clarity, specificity, and context awareness in the prompt instructions. ',
}

Judge = JudgeProxyAgent(
    name="Judge",
    system_message="You are an advisor",
    code_execution_config=False,
    llm_config=llm_config_judge,
)

# groupchat = autogen.GroupChat(
#     agents=[Judge, PromptGenerator, LLM],
#     messages=[],
#     max_round=50,
#     speaker_selection_method="round_robin"
# )

In [38]:
result = Judge.initiate_chat(
    recipient = LLM,
    max_turns = 2,
    message = phoMT_dev_envi[4]["question"],
    answer = phoMT_dev_envi[4]["answer"]
)

Judge (to LLM):

Translate a sentence from English to Vietnamese. Produce an accurate, context-sensitive translation that maintains the tone and meaning of the original sentence. Ensure that the output sounds natural for native Vietnamese speakers.
    Respond only with the requested output. Do not include any explanations, introductions, follow-up remarks, or additional feedback. Provide exactly and only what is specified in the task.
    The text:
    At this time, there have been no reported injuries among the 46 publishers in the two congregations on Great Abaco Island.

--------------------------------------------------------------------------------
LLM (to Judge):

Tới giờ, chưa có báo cáo về nạn nhân thương tật nào trong số 46 nhà xuất bản tại hai giáo xứ trên đảo Abaco Đại.

--------------------------------------------------------------------------------


GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
Predicting DataLoader 0: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:04<00:00,  4.84s/it]


Score: 0.5986794233322144
Judge (to LLM):

Differences:

* Tới giờ (Until now) vs Theo báo cáo đến thời điểm hiện tại (According to the report up to this point)
* nạn nhân thương tật (casualty) vs không có anh chị nào bị thương (no one was injured)
* hai giáo xứ (two churches) vs hai hội thánh (two congregations)

Guidance:

* Use Theo báo cáo đến thời điểm hiện tại instead of Tới giờ to maintain the same timing perspective as the original sentence
* Replace nạn nhân thương tật with không có anh chị nào bị thương to match the original phrase's meaning and tone
* Change hai giáo xứ to hai hội thánh to align with the original term for "publishers" being people, not churches

--------------------------------------------------------------------------------
LLM (to Judge):

Theo báo cáo đến thời điểm hiện tại, chưa có báo cáo về không có anh chị nào bị thương trong số 46 nhà xuất bản tại hai hội thánh trên đảo Abaco Đại.

---------------------------------------------------------------------

In [40]:
Judge.chat_messages_for_summary(LLM)

[{'content': 'Translate a sentence from English to Vietnamese. Produce an accurate, context-sensitive translation that maintains the tone and meaning of the original sentence. Ensure that the output sounds natural for native Vietnamese speakers.\n    Respond only with the requested output. Do not include any explanations, introductions, follow-up remarks, or additional feedback. Provide exactly and only what is specified in the task.\n    The text:\n    At this time, there have been no reported injuries among the 46 publishers in the two congregations on Great Abaco Island.',
  'role': 'assistant'},
 {'content': 'Analyze the original sentence: At this time, there have been no reported injuries among the 46 publishers in the two congregations on Great Abaco Island., the expected Vietnamese translation: Theo báo cáo đến thời điểm hiện tại, trong 46 người công bố thuộc hai hội thánh ở đảo Great Abaco thì không có anh chị nào bị thương., and the generated translation: Tới giờ, chưa có báo 

In [46]:
print(result)

Tới giờ, chưa có báo cáo về nạn nhân thương tật nào trong số 46 nhà xuất bản tại hai giáo xứ trên đảo Abaco Đại.


In [20]:
judge_result = "Theo báo cáo đến thời điểm hiện tại, trong 46 người công bố thuộc hai hội thánh ở đảo Great Abaco thì không có nạn nhân bị thương."
get_score(phoMT_dev_envi[4]["question"], judge_result, phoMT_dev_envi[4]["answer"])

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
Predicting DataLoader 0: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:04<00:00,  4.83s/it]


0.8418942093849182

### Agent pairing

In [43]:
def score_translate(message, answer, turns = 1):
    result = Judge.initiate_chat(
        recipient = LLM,
        max_turns = turns + 1,
        message = message,
        answer = answer
    )
    return get_score(message, str(result), answer)

In [14]:
score_dev = []
for i in range(10):
    x = phoMT_dev_envi[i]
    score_dev.append(score_translate(x['question'], x['answer'], 0))
score_test = []
for i in range(10):
    x = phoMT_test_envi[i]
    score_test.append(score_translate(x['question'], x['answer'], 0))

Judge (to LLM):

Translate a sentence from English to Vietnamese. Produce an accurate, context-sensitive translation that maintains the tone and meaning of the original sentence. Ensure that the output sounds natural for native Vietnamese speakers.
    Respond only with the requested output. Do not include any explanations, introductions, follow-up remarks, or additional feedback. Provide exactly and only what is specified in the task.
    The text:
    Hurricane Dorian, one of the most powerful storms ever recorded in the Atlantic Ocean, made landfall as a Category 5 storm on Great Abaco Island in the northern Bahamas on Sunday morning, September 1, 2019.

--------------------------------------------------------------------------------
LLM (to Judge):

Bão Dorian, một cơn bão mạnh nhất được ghi nhận trong Đại Tây Dương, đổ bộ như một cơn bão hạng 5 vào đảo Abaco lớn ở miền bắc Bahamas vào sáng ngày 1 tháng 9 năm 2019.

------------------------------------------------------------------

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
Predicting DataLoader 0: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:04<00:00,  4.22s/it]


Judge (to LLM):

Translate a sentence from English to Vietnamese. Produce an accurate, context-sensitive translation that maintains the tone and meaning of the original sentence. Ensure that the output sounds natural for native Vietnamese speakers.
    Respond only with the requested output. Do not include any explanations, introductions, follow-up remarks, or additional feedback. Provide exactly and only what is specified in the task.
    The text:
    Dorian is especially dangerous due to its slow movement, high wind speeds, and heavy rains.

--------------------------------------------------------------------------------
LLM (to Judge):

Dorian đặc biệt nguy hiểm do sự di chuyển chậm, tốc độ gió cao và mưa nặng.

--------------------------------------------------------------------------------


GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
Predicting DataLoader 0: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:03<00:00,  3.19s/it]


Judge (to LLM):

Translate a sentence from English to Vietnamese. Produce an accurate, context-sensitive translation that maintains the tone and meaning of the original sentence. Ensure that the output sounds natural for native Vietnamese speakers.
    Respond only with the requested output. Do not include any explanations, introductions, follow-up remarks, or additional feedback. Provide exactly and only what is specified in the task.
    The text:
    The storm passed by the Leeward Islands, Puerto Rico, and the Virgin Islands as a tropical storm with little or no reported damage.

--------------------------------------------------------------------------------
LLM (to Judge):

Bão đã qua các đảo gió sau, Puerto Rico và Các đảo Thánh như một cơn bão nhiệt đới với ít hay không cóรายงาน về thiệt hại nào.

--------------------------------------------------------------------------------


...

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
Predicting DataLoader 0: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:03<00:00,  3.65s/it]


Judge (to LLM):

Translate a sentence from English to Vietnamese. Produce an accurate, context-sensitive translation that maintains the tone and meaning of the original sentence. Ensure that the output sounds natural for native Vietnamese speakers.
    Respond only with the requested output. Do not include any explanations, introductions, follow-up remarks, or additional feedback. Provide exactly and only what is specified in the task.
    The text:
    International government agencies and officials have responded to Russia's Supreme Court decision that criminalizes the worship of Jehovah's Witnesses in Russia.

--------------------------------------------------------------------------------
LLM (to Judge):

Các cơ quan và chính quyền quốc tế đã phản ứng đối với quyết định của Tòa án Tối cao Nga rằng hành xử thờ cúng Jehovah trong Nga là tội phạm.

--------------------------------------------------------------------------------


GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
Predicting DataLoader 0: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:03<00:00,  3.86s/it]


Judge (to LLM):

Translate a sentence from English to Vietnamese. Produce an accurate, context-sensitive translation that maintains the tone and meaning of the original sentence. Ensure that the output sounds natural for native Vietnamese speakers.
    Respond only with the requested output. Do not include any explanations, introductions, follow-up remarks, or additional feedback. Provide exactly and only what is specified in the task.
    The text:
    These statements have criticized Russia's unjust and harsh judicial action against a minority religious group known for peaceful religious activity.

--------------------------------------------------------------------------------
LLM (to Judge):

Các tuyên bố này đã phê phán hành động pháp lý không công bằng và khắc nghiệt của Nga đối với một nhóm tôn giáo thiểu số được biết đến với hoạt động tôn giáo hòa bình.

(Note: Please keep in mind that translation quality may vary depending on the context and nuances of the original sentence. T

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
Predicting DataLoader 0: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:03<00:00,  3.75s/it]


In [15]:
numpy.average(score_dev)

0.4482489854097366

In [16]:
score_dev

[0.3027490973472595,
 0.6770807504653931,
 0.44531580805778503,
 0.3579513132572174,
 0.3298579752445221,
 0.38401174545288086,
 0.34956368803977966,
 0.3959449231624603,
 0.845365047454834,
 0.3946495056152344]

In [17]:
numpy.average(score_test)

0.4235638022422791

In [18]:
score_test

[0.38328737020492554,
 0.422347754240036,
 0.6424505710601807,
 0.5040085315704346,
 0.32535678148269653,
 0.4432627856731415,
 0.28456687927246094,
 0.41430363059043884,
 0.3621178865432739,
 0.453935831785202]

In [19]:
score_dev = []
for i in range(10):
    x = phoMT_dev_envi[i]
    score_dev.append(score_translate(x['question'], x['answer'], 1))
score_test = []
for i in range(10):
    x = phoMT_test_envi[i]
    score_test.append(score_translate(x['question'], x['answer'], 1))

Judge (to LLM):

Translate a sentence from English to Vietnamese. Produce an accurate, context-sensitive translation that maintains the tone and meaning of the original sentence. Ensure that the output sounds natural for native Vietnamese speakers.
    Respond only with the requested output. Do not include any explanations, introductions, follow-up remarks, or additional feedback. Provide exactly and only what is specified in the task.
    The text:
    Hurricane Dorian, one of the most powerful storms ever recorded in the Atlantic Ocean, made landfall as a Category 5 storm on Great Abaco Island in the northern Bahamas on Sunday morning, September 1, 2019.

--------------------------------------------------------------------------------
LLM (to Judge):

Bão Dorian, một cơn bão mạnh nhất được ghi nhận trong Đại Tây Dương, đổ bộ như một cơn bão hạng 5 vào đảo Abaco lớn ở miền bắc Bahamas vào sáng ngày 1 tháng 9 năm 2019.

------------------------------------------------------------------

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
Predicting DataLoader 0: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:05<00:00,  5.35s/it]


Score: 0.9390031099319458
Judge (to LLM):

The result is passable. Please reply me with the same answer as before.

--------------------------------------------------------------------------------


HTTP Request: POST http://localhost:11434/v1/chat/completions "HTTP/1.1 200 OK"


[autogen.oai.client: 11-18 02:00:44] {329} WARNING - Model llama3 is not found. The cost will be 0. In your config_list, add field {"price" : [prompt_price_per_1k, completion_token_price_per_1k]} for customized pricing.


...

HTTP Request: POST http://localhost:11434/v1/chat/completions "HTTP/1.1 200 OK"


[autogen.oai.client: 11-18 02:11:44] {329} WARNING - Model llama3 is not found. The cost will be 0. In your config_list, add field {"price" : [prompt_price_per_1k, completion_token_price_per_1k]} for customized pricing.


Model llama3 is not found. The cost will be 0. In your config_list, add field {"price" : [prompt_price_per_1k, completion_token_price_per_1k]} for customized pricing.


LLM (to Judge):

I apologize for the differences between my generated translation and the original. Your revised translation is an excellent improvement, as it maintains the same tone, style, and nuance as the original sentence while still conveying the intended meaning in Vietnamese. Your suggested translation will certainly be more natural-sounding to native Vietnamese speakers:

"Các lời nhận xét đã chỉ trích nước Nga có hành động tư pháp khắc nghiệt và bất công nhắm vào một nhóm tôn giáo nhỏ được biết đến là hoạt động một cách ôn hoà."

Thank you for providing constructive feedback that allows me to improve my translations. I will keep this in mind and strive to produce more accurate and context-sensitive translations moving forward.

--------------------------------------------------------------------------------


GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
Predicting DataLoader 0: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:05<00:00,  5.42s/it]


In [20]:
numpy.average(score_dev)

0.7821441501379013

In [21]:
score_dev

[0.9390031099319458,
 1.0,
 0.7772258520126343,
 0.9374867677688599,
 0.5986794233322144,
 0.4168749749660492,
 0.6661844849586487,
 0.9230018258094788,
 1.0,
 0.5629850625991821]

In [22]:
numpy.average(score_test)

0.7875484570860862

In [23]:
score_test

[0.7367154359817505,
 0.9583954811096191,
 0.9900977611541748,
 0.898392915725708,
 0.20618979632854462,
 0.9950833320617676,
 0.5456974506378174,
 0.9136548042297363,
 0.8888406157493591,
 0.7424169778823853]

## Improve

In [26]:
llm_config

{'config_list': [{'model': 'llama3',
   'base_url': 'http://localhost:11434/v1',
   'api_key': 'ollama'}]}

In [45]:
EPOCH = 1
# optimizer_model = "gpt-4-1106-preview"
optimizer = AgentOptimizer(max_actions_per_step=3, llm_config=llm_config, optimizer_model="llama3")
for i in range(EPOCH):
    # for index, query in enumerate(train_data):
    for index in range(15):
        query = phoMT_train_envi[index]
        # is_correct = user_proxy.initiate_chat(assistant, answer=query["answer"], problem=query["question"])
        result = score_translate(query['question'], query['answer'], 1)
        history = Judge.chat_messages_for_summary(LLM)
        print(history)
        optimizer.record_one_conversation(history, is_satisfied=True)
    register_for_llm, register_for_exector = optimizer.step()
    for item in register_for_llm:
        LLM.update_function_signature(**item)
    if len(register_for_exector.keys()) > 0:
        Judge.register_function(function_map=register_for_exector)

Judge (to LLM):

Translate a sentence from English to Vietnamese. Produce an accurate, context-sensitive translation that maintains the tone and meaning of the original sentence. Ensure that the output sounds natural for native Vietnamese speakers.
    Respond only with the requested output. Do not include any explanations, introductions, follow-up remarks, or additional feedback. Provide exactly and only what is specified in the task.
    The text:
    It begins with a countdown.

--------------------------------------------------------------------------------
LLM (to Judge):

Nó bắt đầu với một giờ đếm ngược.

--------------------------------------------------------------------------------


GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
Predicting DataLoader 0: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:03<00:00,  3.37s/it]


Score: 0.8480406999588013
Judge (to LLM):

Differences:

* Nó bắt đầu với một giờ đếm ngược. uses "một giờ" (one hour) instead of "buổi lễ" (countdown ceremony/event), implying a literal countdown time rather than an event.
* Tone: The original sentence has a more dramatic tone, using "it begins with" to introduce the story, while Nó bắt đầu với một giờ đếm ngược. focuses on the timing aspect.

Guidance:

* Replace "một giờ" with "buổi lễ" to align with the original meaning of "countdown ceremony/event".
* Retain the phrase "Nó bắt đầu" to maintain a similar tone and style to the original sentence.
* Adjusted translation: Nó bắt đầu với buổi lễ đếm ngược. (preserves tone, style, and naturalness in Vietnamese while addressing discrepancies)

--------------------------------------------------------------------------------
LLM (to Judge):

Nó bắt đầu với buổi lễ đếm ngược.

--------------------------------------------------------------------------------


GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
Predicting DataLoader 0: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:03<00:00,  3.43s/it]


[{'content': 'Translate a sentence from English to Vietnamese. Produce an accurate, context-sensitive translation that maintains the tone and meaning of the original sentence. Ensure that the output sounds natural for native Vietnamese speakers.\n    Respond only with the requested output. Do not include any explanations, introductions, follow-up remarks, or additional feedback. Provide exactly and only what is specified in the task.\n    The text:\n    It begins with a countdown.', 'role': 'assistant'}, {'content': 'Analyze the original sentence: It begins with a countdown., the expected Vietnamese translation: Câu chuyện bắt đầu với buổi lễ đếm ngược., and the generated translation: Nó bắt đầu với một giờ đếm ngược.. Identify the differences between Nó bắt đầu với một giờ đếm ngược. and Câu chuyện bắt đầu với buổi lễ đếm ngược., and provide guidance to improve the translation so it aligns more closely with Câu chuyện bắt đầu với buổi lễ đếm ngược.. Focus on preserving meaning, tone, 

...

Model llama3 is not found. The cost will be 0. In your config_list, add field {"price" : [prompt_price_per_1k, completion_token_price_per_1k]} for customized pricing.


LLM (to Judge):

Tự độ cũng đồng thời với chia cắt, khi Ấn Độ do Anh quản lý bị chia thành hai nước là Ấn Độ và Pakistan.

--------------------------------------------------------------------------------


GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
Predicting DataLoader 0: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:04<00:00,  4.20s/it]
HTTP Request: POST http://localhost:11434/v1/chat/completions "HTTP/1.1 400 Bad Request"


[{'content': 'Translate a sentence from English to Vietnamese. Produce an accurate, context-sensitive translation that maintains the tone and meaning of the original sentence. Ensure that the output sounds natural for native Vietnamese speakers.\n    Respond only with the requested output. Do not include any explanations, introductions, follow-up remarks, or additional feedback. Provide exactly and only what is specified in the task.\n    The text:\n    Independence also coincided with Partition, which divided British - controlled India into the two nations of India and Pakistan.', 'role': 'assistant'}, {'content': 'Analyze the original sentence: Independence also coincided with Partition, which divided British - controlled India into the two nations of India and Pakistan., the expected Vietnamese translation: Độc lập đồng thời tạo ra sự chia cắt, khi Ấn Độ thuộc địa của Anh bị chia thành hai quốc gia có chủ quyền là Ấn Độ và Pakistan., and the generated translation: Tự độ cũng trùng h

BadRequestError: Error code: 400 - {'error': {'message': 'llama3 does not support tools', 'type': 'api_error', 'param': None, 'code': None}}

## Compare

In [46]:
score_test = []
for i in range(10):
    x = phoMT_test_envi[i]
    score_test.append(score_translate(x['question'], x['answer'], 0))

Judge (to LLM):

Translate a sentence from English to Vietnamese. Produce an accurate, context-sensitive translation that maintains the tone and meaning of the original sentence. Ensure that the output sounds natural for native Vietnamese speakers.
    Respond only with the requested output. Do not include any explanations, introductions, follow-up remarks, or additional feedback. Provide exactly and only what is specified in the task.
    The text:
    Brother Albert Barnett and his wife, Sister Susan Barnett, from the West Congregation in Tuscaloosa, Alabama

--------------------------------------------------------------------------------
LLM (to Judge):

Chịu Brother Albert Barnett và vợ ông, Chị Susan Barnett, từ Đình Tây ở Tuscaloosa, Alabama.

--------------------------------------------------------------------------------


GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
Predicting DataLoader 0: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:03<00:00,  3.84s/it]


Judge (to LLM):

Translate a sentence from English to Vietnamese. Produce an accurate, context-sensitive translation that maintains the tone and meaning of the original sentence. Ensure that the output sounds natural for native Vietnamese speakers.
    Respond only with the requested output. Do not include any explanations, introductions, follow-up remarks, or additional feedback. Provide exactly and only what is specified in the task.
    The text:
    Severe storms ripped through parts of the southern and midwestern United States on January 11 and 12, 2020.

--------------------------------------------------------------------------------
LLM (to Judge):

Cơn bão cường độ cao đã qua những khu vực phía nam và miền trung Hoa Kỳ vào ngày 11 và 12 tháng 1 năm 2020.

--------------------------------------------------------------------------------


GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
Predicting DataLoader 0: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:03<00:00,  3.95s/it]


Judge (to LLM):

Translate a sentence from English to Vietnamese. Produce an accurate, context-sensitive translation that maintains the tone and meaning of the original sentence. Ensure that the output sounds natural for native Vietnamese speakers.
    Respond only with the requested output. Do not include any explanations, introductions, follow-up remarks, or additional feedback. Provide exactly and only what is specified in the task.
    The text:
    Two days of heavy rain, high winds, and numerous tornadoes caused major damage across multiple states.

--------------------------------------------------------------------------------
LLM (to Judge):

Hai ngày mưa nặng, gió mạnh và nhiều cơn xoáy trời đã gây thiệt hại lớn trên nhiều bang.

--------------------------------------------------------------------------------


...

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
Predicting DataLoader 0: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:03<00:00,  3.90s/it]


Judge (to LLM):

Translate a sentence from English to Vietnamese. Produce an accurate, context-sensitive translation that maintains the tone and meaning of the original sentence. Ensure that the output sounds natural for native Vietnamese speakers.
    Respond only with the requested output. Do not include any explanations, introductions, follow-up remarks, or additional feedback. Provide exactly and only what is specified in the task.
    The text:
    International government agencies and officials have responded to Russia's Supreme Court decision that criminalizes the worship of Jehovah's Witnesses in Russia.

--------------------------------------------------------------------------------
LLM (to Judge):

Các cơ quan và chính quyền quốc tế đã phản ứng đối với quyết định của Tòa án Tối cao Nga rằng hành xử thờ cúng Jehovah trong Nga là tội phạm.

--------------------------------------------------------------------------------


GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
Predicting DataLoader 0: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:04<00:00,  4.13s/it]


Judge (to LLM):

Translate a sentence from English to Vietnamese. Produce an accurate, context-sensitive translation that maintains the tone and meaning of the original sentence. Ensure that the output sounds natural for native Vietnamese speakers.
    Respond only with the requested output. Do not include any explanations, introductions, follow-up remarks, or additional feedback. Provide exactly and only what is specified in the task.
    The text:
    These statements have criticized Russia's unjust and harsh judicial action against a minority religious group known for peaceful religious activity.

--------------------------------------------------------------------------------
LLM (to Judge):

Các tuyên bố này đã phê phán hành động pháp lý không công bằng và khắc nghiệt của Nga đối với một nhóm tôn giáo thiểu số được biết đến với hoạt động tôn giáo hòa bình.

(Note: Please keep in mind that translation quality may vary depending on the context and nuances of the original sentence. T

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
Predicting DataLoader 0: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:03<00:00,  3.99s/it]


In [47]:
numpy.average(score_test)

0.4235638022422791

In [48]:
score_test

[0.38328737020492554,
 0.422347754240036,
 0.6424505710601807,
 0.5040085315704346,
 0.32535678148269653,
 0.4432627856731415,
 0.28456687927246094,
 0.41430363059043884,
 0.3621178865432739,
 0.453935831785202]